In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()
%matplotlib inline

pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 999
from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

#------------------------------------------------------------------------------


train = pd.read_csv('../input/train_2016_v2.csv', parse_dates=["transactiondate"])
prop = pd.read_csv('../input/properties_2016.csv')
sample = pd.read_csv('../input/sample_submission.csv')

##### PROCESS DATA FOR REGRESSION

print( "\nProcessing data for REGRESSION ..." )
for c, dtype in zip(prop.columns, prop.dtypes):	
    if dtype == np.float64:		
        prop[c] = prop[c].astype(np.float32)

df_train = train.merge(prop, how='left', on='parcelid')


# drop out ouliers
ax = sns.boxplot(y=df_train["logerror"])


def IQR(x):
    return np.percentile(x, 75) - np.percentile(x, 25)

upper = np.percentile(df_train.logerror.values, 75)+1.5*IQR(df_train.logerror)
lower = np.percentile(df_train.logerror.values, 25)-1.5*IQR(df_train.logerror)

print ("upper,lower" ,upper, lower)


#Subset the data after removing the outliers
df_train=df_train[(df_train['logerror'] < upper) & (df_train['logerror'] > lower)]
df_train.shape

#------------------------------------------------------------------------------
#Feature Engineering
#------------------------------------------------------------------------------
###Property age in years
df_train['D_Prop_Age'] = 2018 - df_train['yearbuilt']   


#Total number of rooms
df_train['D_TotalRooms'] = df_train['bathroomcnt']*df_train['bedroomcnt']

#Missing Count
df_train['D_miss_count']=df_train.apply(lambda x: sum(x.isnull().values), axis = 1) # For rows
#------------------------------------------------------------------------------


count = 0
for col in list(df_train):
    if (len(df_train[col].unique())==1):
        print(col)
        count=count+1
print(count)        



df_train['assessmentyear'].value_counts()

#Find out the missing values by columnwise
df_train.apply(lambda x: sum(x.isnull().values), axis = 0) # For columns

df_train['D_miss_count']=df_train.apply(lambda x: sum(x.isnull().values), axis = 1) # For rows

df_train=df_train[df_train['D_miss_count']<30]
df_train.shape


x_train = df_train.drop(['parcelid', 'logerror', 'transactiondate', 'propertyzoningdesc', 'propertycountylandusecode','assessmentyear'], axis=1)

y_train = df_train['logerror'].values
print(x_train.shape, y_train.shape)

x_train = x_train.apply(lambda x:x.fillna(x.value_counts().index[0]))

x_train.describe()
#x_train=np.nan_to_num(x_train)
#from sklearn.model_selection import train_test_split
#X_train,X_valid,Y_train,Y_valid=train_test_split(x_train,y_train,test_size=0.2,random_state=42)



from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import gc
from sklearn.linear_model import LinearRegression
import random
import datetime as dt


#------------------------------------------------------------------------------
train_columns = x_train.columns

for c in x_train.dtypes[x_train.dtypes == object].index.values:
    x_train[c] = (x_train[c] == True)

del df_train; gc.collect()

split = 90000
x_train, y_train, x_valid, y_valid = x_train[:split], y_train[:split], x_train[split:], y_train[split:]
x_train = x_train.values.astype(np.float32, copy=False)
x_valid = x_valid.values.astype(np.float32, copy=False)

d_train = lgb.Dataset(x_train, label=y_train)
d_valid = lgb.Dataset(x_valid, label=y_valid)

params = {}
params['learning_rate'] = 0.002
params['boosting_type'] = 'gbdt'
params['objective'] = 'regression'
params['metric'] = 'mae'
params['sub_feature'] = 0.5
params['num_leaves'] = 60
params['min_data'] = 500
params['min_hessian'] = 1

watchlist = [d_valid]
clf = lgb.train(params, d_train, 500, watchlist)

del d_train, d_valid; gc.collect()
del x_train, x_valid; gc.collect()

print("Prepare for the prediction ...")
sample = pd.read_csv('../input/sample_submission.csv')
sample['parcelid'] = sample['ParcelId']
df_test = sample.merge(prop, on='parcelid', how='left')
del sample, prop; gc.collect()

#------------------------------------------------------------------------------
###Property age in years
df_test['D_Prop_Age'] = 2018 - df_test['yearbuilt']   


#Total number of rooms
df_test['D_TotalRooms'] = df_test['bathroomcnt']*df_test['bedroomcnt']

#Missing Count
df_test['D_miss_count']=df_test.apply(lambda x: sum(x.isnull().values), axis = 1) # For rows
#------------------------------------------------------------------------------
x_test = df_test[train_columns]
del df_test; gc.collect()
for c in x_test.dtypes[x_test.dtypes == object].index.values:
    x_test[c] = (x_test[c] == True)
x_test = x_test.values.astype(np.float32, copy=False)

print("Start prediction ...")
# num_threads > 1 will predict very slow in kernal
clf.reset_parameter({"num_threads":1})
p_test = clf.predict(x_test)

del x_test; gc.collect()

print("Start write result ...")
sub = pd.read_csv('sample_submission.csv')
for c in sub.columns[sub.columns != 'ParcelId']:
    sub[c] = p_test

sub.to_csv('lgb_starter.csv', index=False, float_format='%.4f')


# Any results you write to the current directory are saved as output.

